In [2]:
!pip install optimum-quanto
#restart kernal after installing
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 58.3 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━

In [3]:
!pip install torch-pruning 
import torch_pruning as tp
from optimum.quanto import Calibration, QTensor, freeze, qfloat8, qint4, qint8, qint2,quantize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.1 MB/s eta 0:00:00eta 0:00:01


/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
def train_model(model,
                train_loader,
                test_loader,
                device,
                learning_rate=1e-1,
                num_epochs=200):

    criterion = nn.CrossEntropyLoss()

    model.to(device)

    optimizer = optim.SGD(model.parameters(),
                          lr=learning_rate,
                          momentum=0.9,
                          weight_decay=1e-4)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                     milestones=[100, 150],
                                                     gamma=0.1,
                                                     last_epoch=-1)

    # Evaluation
    model.eval()
    eval_loss, eval_accuracy = evaluate_model(model=model,
                                              test_loader=test_loader,
                                              device=device,
                                              criterion=criterion)
    print("Epoch: {:03d} Eval Loss: {:.3f} Eval Acc: {:.3f}".format(
        0, eval_loss, eval_accuracy))

    for epoch in range(num_epochs):

        # Training
        model.train()

        running_loss = 0
        running_corrects = 0

        for inputs, labels in train_loader:

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = running_corrects / len(train_loader.dataset)

        # Evaluation
        model.eval()
        eval_loss, eval_accuracy = evaluate_model(model=model,
                                                  test_loader=test_loader,
                                                  device=device,
                                                  criterion=criterion)
        
        scheduler.step()

        print(
            "Epoch: {:03d} Train Loss: {:.3f} Train Acc: {:.3f} Eval Loss: {:.3f} Eval Acc: {:.3f}"
            .format(epoch + 1, train_loss, train_accuracy, eval_loss,
                    eval_accuracy))

    return model

def evaluate_model(model, test_loader, device, criterion=None):

    model.eval()
    model.to(device)

    running_loss = 0
    running_corrects = 0

    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0

        # statistics
        running_loss += loss * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    eval_loss = running_loss / len(test_loader.dataset)
    eval_accuracy = running_corrects / len(test_loader.dataset)

    return eval_loss, eval_accuracy


In [5]:
def save_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.save(model.state_dict(), model_filepath)

def load_model(model, model_filepath, device):

    model.load_state_dict(torch.load(model_filepath, map_location=device))

    return model

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to 32x32 as LeNet-5 expects this input size
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
batch_size = 256
# CIFAR-10 dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 34405529.98it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1017143.55it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9791830.29it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2530286.73it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [7]:
import torch.nn.functional as F
import time
def test(model, device, test_loader):
    model.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        start = time.time()
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            if isinstance(output, QTensor):
                output = output.dequantize()
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        end = time.time()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set evaluated in {:.2f} s: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            end - start, test_loss, correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

model = LeNet5(10)

model = train_model(model=model,
                        train_loader=train_loader,
                        test_loader=test_loader,
                        device=device,
                        learning_rate=1e-1,
                        num_epochs=1)

Epoch: 000 Eval Loss: 2.310 Eval Acc: 0.075
Epoch: 001 Train Loss: 0.302 Train Acc: 0.913 Eval Loss: 0.052 Eval Acc: 0.985


In [9]:
test(model, device, test_loader)

model_dir = "/kaggle/working/models/"
model_filename = "LeNet5.pth"
model_filepath = model_dir + model_filename
import os
save_model(model=model, model_dir=model_dir, model_filename=model_filename)


Test set evaluated in 2.83 s: Average loss: -11.0638, Accuracy: 9846/10000 (98%)



In [10]:
all_images = []
targets = []

for image, target in test_loader:
    all_images.append(image)
    targets.append(target)
    break
# Concatenate all the images into a single tensor
all_images_tensor = torch.cat(all_images, dim=0)
targets_tensor = torch.cat(all_images, dim=0)
print(all_images_tensor.shape)
print(targets_tensor.shape)

torch.Size([256, 1, 32, 32])
torch.Size([256, 1, 32, 32])


In [11]:
# example_inputs = torch.randn(1,3,224,224).to(device)

criterion = nn.CrossEntropyLoss()

example_inputs = all_images_tensor.clone().detach().requires_grad_(True).to(device)
eample_targets = targets_tensor.clone().detach().requires_grad_(True).to(device)

# 1. Importance criterion
imp = tp.importance.GroupTaylorImportance() # or GroupNormImportance(p=2), GroupHessianImportance(), etc.

# 2. Initialize a pruner with the model and the importance criterion
ignored_layers = []
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 10:
        ignored_layers.append(m) # DO NOT prune the final classifier!

pruner = tp.pruner.MetaPruner( # We can always choose MetaPruner if sparse training is not required.
    model,
    example_inputs,
    importance=imp,
    #pruning_ratio=0.5, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    pruning_ratio_dict = {model.layer1: 0, model.layer2: 0.5,model.fc: 0.2,model.fc1: 0.6}, # customized pruning ratios for layers or blocks
    ignored_layers=ignored_layers,
)

# 3. Prune & finetune the model
base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_inputs)
if isinstance(imp, tp.importance.GroupTaylorImportance):
    # Taylor expansion requires gradients for importance estimation
#     loss = model(example_inputs).sum() # A dummy loss, please replace this line with your loss function and data!
    loss = criterion(example_inputs,eample_targets)
    loss.backward() # before pruner.step()

In [12]:
pruner.step()

macs, nparams = tp.utils.count_ops_and_params(model.to(device), example_inputs.to(device))
print(
    " Params: %.2f M => %.2f M"
    % (base_nparams / 1e6, nparams / 1e6)
)

test(model, device, test_loader)

torch.save(model,"/kaggle/working/models/pruned.pth")

 Params: 0.06 M => 0.02 M

Test set evaluated in 2.91 s: Average loss: -6.5733, Accuracy: 9259/10000 (93%)



In [14]:
def Quantize(model,weights_dtype,activations_dtype):
    quantize(model, weights=weights_dtype, activations=activations_dtype)
    print("Calibrating ...")
    with Calibration():
        test(model, device, test_loader)
    freeze(model)

weights = [qint8,qint4,qint2,qfloat8]
strs = ["int8.pth","int4.pth","int2.pth","float8.pth"]
activations = [qint8,qint4,qint2,qfloat8]

i= 0 #change here
pruned_model = torch.load("/kaggle/working/models/pruned.pth")
Quantize(model = pruned_model,weights_dtype=weights[i],activations_dtype=activations[i])
path = "/kaggle/working/models/"+strs[i]
torch.save(model,path)

Calibrating ...

Test set evaluated in 4.61 s: Average loss: -6.5475, Accuracy: 9255/10000 (93%)



In [15]:
if i==0:
    ptq_model = torch.load("/kaggle/working/models/int8.pth")
    print("int8 model:")
    test(ptq_model, device, test_loader)
elif i==1:
    ptq_model = torch.load("/kaggle/working/models/int4.pth")
    print("int4 model:")
    test(ptq_model, device, test_loader)
elif i==2:
    ptq_model = torch.load("/kaggle/working/models/int2.pth")
    print("int2 model:")
#     test(ptq_model, device, test_loader)
else:
    ptq_model = torch.load("/kaggle/working/models/float8.pth")
    print("float8 model:")
    test(ptq_model, device, test_loader)

int8 model:

Test set evaluated in 2.86 s: Average loss: -6.5733, Accuracy: 9259/10000 (93%)



In [ ]:
# for name, param in ptq_model.named_parameters():
#     print(name)
#     print(param)
#     break

# **QAT for model(optional)**

In [16]:
quantize(ptq_model,weights=qint8,activations=qint8)
qat_model = train_model(model=ptq_model,
                        train_loader=train_loader,
                        test_loader=test_loader,
                        device=device,
                        learning_rate=1e-1,
                        num_epochs=3)

test(qat_model, device, test_loader)

freeze(qat_model)
print("after freezing:")
test(qat_model, device, test_loader)
torch.save(qat_model,path)

Epoch: 000 Eval Loss: 0.778 Eval Acc: 0.777
Epoch: 001 Train Loss: 0.098 Train Acc: 0.970 Eval Loss: 0.062 Eval Acc: 0.979
Epoch: 002 Train Loss: 0.066 Train Acc: 0.979 Eval Loss: 0.056 Eval Acc: 0.982
Epoch: 003 Train Loss: 0.060 Train Acc: 0.981 Eval Loss: 0.072 Eval Acc: 0.977

Test set evaluated in 3.14 s: Average loss: -13.2942, Accuracy: 9771/10000 (98%)

after freezing:

Test set evaluated in 3.08 s: Average loss: -13.2942, Accuracy: 9771/10000 (98%)



In [17]:
for name, param in qat_model.named_parameters():
    print(name)
    print(param)
    break

layer1.0.weight
QBytesTensor(tensor([[[[  14, -105,  -63,  -20,   52],
          [  17,  -59,  -73,  -45,   66],
          [  -4,  -74,  -99,  -18,   35],
          [  12,  -36,  -84,  -52,   34],
          [  49,  -58, -127,  -65,   12]]],


        [[[ -19,  -48,   -2,   54,   82],
          [  27,  -49,  -95,  -62,    5],
          [  61,  -65,  -86,  -88,  -59],
          [  88,   41,  -40,  -30,  -49],
          [  28,  127,  125,   84,   40]]],


        [[[-127,  -82,  -58,  -20,   -3],
          [ -41,  -79,  -85,  -89,  -69],
          [   7,   14,  -10,  -37,  -73],
          [  51,   17,   53,   16,  -11],
          [  14,   10,   -8,    7,   29]]],


        [[[ -22,  -24,  -54,  -55,   -1],
          [ -11,  -84,  -91,  -61,   96],
          [-127,  -91,  -24,   26,   95],
          [-125,  -33,   35,   74,  104],
          [  50,   84,   82,  120,   34]]],


        [[[  38,   37,  -53, -104,  -88],
          [  36,    8,  -70,  -42,  -69],
          [  41,  -22,  -68, -1